In [10]:
from pydub import AudioSegment

video = AudioSegment.from_file("../samples/videoplayback.mp4", format="mp4")

In [11]:
audio = video.set_channels(1).set_frame_rate(16000).set_sample_width(2)

In [12]:
audio.export("../samples/out/videoplayback.wav", format="wav")

<_io.BufferedRandom name='../samples/out/videoplayback.wav'>